In [1]:
import os, math
import numpy as np
import pandas as pd
import seaborn as sns
import helpers

%matplotlib inline
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, f_classif
from sklearn.metrics import log_loss

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn import *
import sklearn

//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [40]:
def read_files(file, fileindex):
    return pd.read_csv(file, \
            names = np.array(['feature'+str(i) for i in range(fileindex*9, 9+fileindex*9)]), usecols = np.array(range(1, 10)), skiprows = [0])
    
def create_feature_matrix(file_list):
    pd_list = []
    for i, file in enumerate(file_list):
        pd_list.append(read_files(file, i))
    return pd.concat(pd_list, axis = 1)


In [60]:
X_train=create_feature_matrix(['/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/2nd_layer_data/train_prob_gene_svm_1.16',\
                      '/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/2nd_layer_data/train_prob_tfidf_svm_1.08'])
X_test=create_feature_matrix(['/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/2nd_layer_data/test_prob_gene_svm_1.16',\
                      '/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/2nd_layer_data/test_prob_tfidf_svm_1.08'])
text, variants = helpers.get_training('./input/training_variants', './input/training_text')
y_train = variants['Class']-1
text, variants = helpers.get_test('./input/training_variants', './input/training_text')
y_test = variants['Class']-1

In [72]:
##Fitting
parameters = {
    "estimator__C": [0.1, 1, 10],
    "estimator__kernel": ['linear', 'rbf']
    #"estimator__degree": [2, 3]
}

clf = GridSearchCV(OneVsRestClassifier(svm.SVC(probability=True, class_weight='balanced')),\
                   param_grid=parameters, scoring='neg_log_loss', n_jobs=-1)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [73]:
pd.DataFrame.from_dict(clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_estimator__C,param_estimator__kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,2.732365,0.180094,-0.455738,-0.443291,0.1,linear,"{'estimator__C': 0.1, 'estimator__kernel': 'li...",5,-0.475354,-0.435464,-0.460166,-0.442545,-0.431573,-0.451863,0.045832,0.002981,0.018147,0.006716
1,6.927477,0.490919,-0.539708,-0.524755,0.1,rbf,"{'estimator__C': 0.1, 'estimator__kernel': 'rbf'}",6,-0.533283,-0.514638,-0.547435,-0.523528,-0.538425,-0.536100,0.331286,0.005673,0.005852,0.008805
2,1.342477,0.087662,-0.433231,-0.411119,1,linear,"{'estimator__C': 1, 'estimator__kernel': 'line...",2,-0.451858,-0.401444,-0.428384,-0.411895,-0.419356,-0.420018,0.013084,0.003624,0.013705,0.007602
3,2.823190,0.207675,-0.452666,-0.438756,1,rbf,"{'estimator__C': 1, 'estimator__kernel': 'rbf'}",4,-0.471804,-0.430612,-0.456627,-0.437895,-0.429449,-0.447761,0.025896,0.002606,0.017518,0.007027
4,1.469230,0.076524,-0.442127,-0.406673,10,linear,"{'estimator__C': 10, 'estimator__kernel': 'lin...",3,-0.464150,-0.399339,-0.430602,-0.405113,-0.431531,-0.415568,0.018128,0.003943,0.015612,0.006717
5,1.680560,0.097937,-0.428876,-0.399474,10,rbf,"{'estimator__C': 10, 'estimator__kernel': 'rbf'}",1,-0.447086,-0.389814,-0.425587,-0.401947,-0.413858,-0.406659,0.103885,0.014026,0.013764,0.007096


In [74]:
#Evaluate
y_train_prob = clf.predict_proba(X_train)
y_test_prob = clf.predict_proba(X_test)
print(log_loss(y_train, y_train_prob, eps=1e-15, normalize=True, labels=range(9)))
#print(log_loss(y_val, clf.predict_proba(X_val), eps=1e-15, normalize=True, labels=range(9)))
print(log_loss(y_test, y_test_prob, eps=1e-15, normalize=True, labels=range(9)))

0.398035466254
1.15845494445


In [75]:
clf.score(X_test,y_test)

-1.1584549444483325